In [84]:
#imports
import duckdb
import warnings
import pandas as pd

In [85]:
#Settings
pd.set_option('display.max_columns', None)
warnings.simplefilter(action='ignore', category=FutureWarning) 

In [86]:
# create a connection to a file called 'database.db'
con = duckdb.connect('database.db')

In [87]:
#describe podcasts table
con.sql("describe podcasts")

┌────────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│  column_name   │ column_type │  null   │   key   │ default │ extra │
│    varchar     │   varchar   │ varchar │ varchar │ varchar │ int32 │
├────────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ podcast_id     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ itunes_id      │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ slug           │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ itunes_url     │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ title          │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ author         │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ description    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ average_rating │ DOUBLE      │ YES     │ NULL    │ NULL    │  NULL │
│ ratings_count  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ scraped_at     │ TIMESTAMP   │ YES     │ NULL    │ NULL    │  NULL │
├─────

In [88]:
#describe categories table
con.sql("describe categories")

┌─────────────┬─────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │ column_type │  null   │   key   │ default │ extra │
│   varchar   │   varchar   │ varchar │ varchar │ varchar │ int32 │
├─────────────┼─────────────┼─────────┼─────────┼─────────┼───────┤
│ podcast_id  │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ itunes_id   │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
│ category    │ VARCHAR     │ YES     │ NULL    │ NULL    │  NULL │
└─────────────┴─────────────┴─────────┴─────────┴─────────┴───────┘

In [89]:
#describe reviews table
con.sql("describe reviews")


┌─────────────┬──────────────────────────┬─────────┬─────────┬─────────┬───────┐
│ column_name │       column_type        │  null   │   key   │ default │ extra │
│   varchar   │         varchar          │ varchar │ varchar │ varchar │ int32 │
├─────────────┼──────────────────────────┼─────────┼─────────┼─────────┼───────┤
│ podcast_id  │ VARCHAR                  │ YES     │ NULL    │ NULL    │  NULL │
│ title       │ VARCHAR                  │ YES     │ NULL    │ NULL    │  NULL │
│ content     │ VARCHAR                  │ YES     │ NULL    │ NULL    │  NULL │
│ rating      │ INTEGER                  │ YES     │ NULL    │ NULL    │  NULL │
│ author_id   │ VARCHAR                  │ YES     │ NULL    │ NULL    │  NULL │
│ created_at  │ TIMESTAMP WITH TIME ZONE │ YES     │ NULL    │ NULL    │  NULL │
└─────────────┴──────────────────────────┴─────────┴─────────┴─────────┴───────┘

In [90]:
#Look into data for reviews
con.sql("select * from reviews limit 5")

┌──────────────────────┬──────────────────────┬──────────────────────┬────────┬─────────────────┬──────────────────────┐
│      podcast_id      │        title         │       content        │ rating │    author_id    │      created_at      │
│       varchar        │       varchar        │       varchar        │ int32  │     varchar     │ timestamp with tim…  │
├──────────────────────┼──────────────────────┼──────────────────────┼────────┼─────────────────┼──────────────────────┤
│ 52e3d2c4fab4e80a8b…  │ Argh                 │ What the heck is t…  │      1 │ a23655ce5565f32 │ 2019-04-30 21:24:0…  │
│ 52e3d2c4fab4e80a8b…  │ Miss gross overrat…  │ The Howard stern i…  │      1 │ e5d461c5f91c8d2 │ 2019-06-07 10:25:3…  │
│ 52e3d2c4fab4e80a8b…  │ I love Terri and F…  │ People get really …  │      5 │ 806cc75442c8f3d │ 2019-05-15 05:42:5…  │
│ 52e3d2c4fab4e80a8b…  │ Fresh Air subscrip…  │ I’ve been a subscr…  │      4 │ 3e1e7e8417a3e85 │ 2019-05-05 22:39:1…  │
│ 52e3d2c4fab4e80a8b…  │ Climate

In [91]:
#Total Number of reviews
con.sql("select count(*) as total_number_of_reviews from reviews")

┌─────────────────────────┐
│ total_number_of_reviews │
│          int64          │
├─────────────────────────┤
│                 5607021 │
└─────────────────────────┘

In [92]:
#Unique authors contributing reviews
con.sql("select count(DISTINCT author_id) as unique_authors from reviews")

┌────────────────┐
│ unique_authors │
│     int64      │
├────────────────┤
│        3256501 │
└────────────────┘

In [93]:
authors = con.sql("select author_id,count(author_id) as count from reviews group by author_id")


In [94]:
#Top 5 authors
authors.df().sort_values('count',ascending=False).head(5)

,author_id,count
406113,d3307adeffa285c,1681
2307799,791f376aca3c08d,1303
1341207,8ea31f030185a88,1248
14367,96a3f0358e7808d,1229
2590713,04c3b6defe22fb1,1094


In [95]:
#Bottom 5 authors
authors.df().sort_values('count',ascending=True).head(5)

,author_id,count
1628250,dfec7ad45f7f40f,1
2011084,f088145d2aff6e4,1
2011085,2eb4372941686c1,1
2011086,5ef7e23e55796ab,1
2011087,aba33f9d0d67cc6,1


In [96]:
#Look into data for reviews that have an average rating
rated_podcasts = con.sql("select title,average_rating,ratings_count from podcasts where average_rating >= 0")
rated_podcasts

┌─────────────────────────────────────────────────────────────────────────┬────────────────┬───────────────┐
│                                  title                                  │ average_rating │ ratings_count │
│                                 varchar                                 │     double     │    varchar    │
├─────────────────────────────────────────────────────────────────────────┼────────────────┼───────────────┤
│ Rahdo Talks Through                                                     │            4.6 │ 60            │
│ OriginALL                                                               │            3.7 │ 3             │
│ The Big Podcast With Shaq                                               │            4.6 │ 1.3K          │
│ Galaktacus                                                              │            5.0 │ 20            │
│ No Extra Words one person's search for story                            │            5.0 │ 11            │
│ The Talking Pictu

In [97]:
total_podcast_ratings = 0
rated_podcasts_df = rated_podcasts.df()
podcast_ratings = []
for (colname,colval) in rated_podcasts_df.iteritems():
    if colname == "ratings_count":
        for x in colval:
            if  "K" not in x:
                total_podcast_ratings += int(x)
                podcast_ratings.append(int(x))
            else:
                formatted = float(x[0:x.find('K')])
                total_podcast_ratings += (1000 * formatted)   
                podcast_ratings.append(int(1000 * formatted))
rated_podcasts_df.drop("ratings_count", axis = 1, inplace = True)
rated_podcasts_df["ratings_count"] = podcast_ratings

In [98]:
#First five values in rated podcasts data frame
rated_podcasts_df.head()


,title,average_rating,ratings_count
0,Rahdo Talks Through,4.6,60
1,OriginALL,3.7,3
2,The Big Podcast With Shaq,4.6,1300
3,Galaktacus,5.0,20
4,No Extra Words one person's search for story,5.0,11


In [99]:
#Most rated podcasts
rated_podcasts_df.sort_values('ratings_count',ascending=False).head(5)

,title,average_rating,ratings_count
6022,Pardon My Take,4.9,56000
12424,Pod Save America,4.7,51000
57663,Stuff You Should Know,4.6,39000
12588,Small Town Murder,4.9,31000
14355,Wait Wait...Don't Tell Me!,4.7,21000


In [100]:
#Podcasts with top 5 average rating
rated_podcasts_df.sort_values('average_rating',ascending=False).head(5)

,title,average_rating,ratings_count
41494,Cerealcast,5.0,1
42157,Global Denmark Podcast,5.0,9
40733,Faking It: An SFX Podcast,5.0,4
73774,NSVille,5.0,3
40735,Under The First Floor,5.0,5


In [101]:
#Top rated podcasts with most ratings
rated_podcasts_df.sort_values(['average_rating', 'ratings_count'],
              ascending = [False, False]).head(5)

,title,average_rating,ratings_count
41479,The Corp,5.0,9300
51386,The TryPod,5.0,8500
72481,Grumpy Old Geeks,5.0,6300
25308,The Anthropocene Reviewed,5.0,4100
51435,Cal's Week in Review,5.0,4000


In [102]:
con.sql("select * from categories").df().head(5)

,podcast_id,itunes_id,category
0,943d1c4349913bde09ef82b69f83cfdf,1446474395,education
1,f53f27d03cdc5040ab4fd2ea30e3dd4f,840646770,society-culture
2,49fa5f1097ae9046ce1a0c56e31b533c,1541959455,arts-performing-arts
3,dbe2dc5aa097523aa294a3b659293af4,1146609744,tv-film
4,2349ed9d26f16029cfc69902bf319ca5,1475252651,education


In [103]:
#Unique Categories
con.sql("select count(distinct category) as unique_categories from categories").df().head(5)

,unique_categories
0,110


In [104]:
categories = con.sql("select category,count(category) as count from categories group by category").df()
categories.head()

,category,count
0,news-daily-news,13101
1,spirituality,35792
2,business,211977
3,society-culture,301139
4,health-fitness,132096


In [105]:
#Top 5 most common categories
categories.sort_values('count',ascending=False).head(5)

,category,count
14,education,311924
3,society-culture,301139
15,arts,227713
2,business,211977
6,religion-spirituality,206257


In [106]:
#Bottom 5 most common categories
categories.sort_values('count',ascending=True).head(5)

,category,count
108,sports-volleyball,265
102,sports-swimming,304
92,sports-tennis,563
95,sports-rugby,648
89,sports-cricket,755
